# Create Tilesheet for Font Charset

This creates latex figures that act as tilesheets for each of the planets.

First we load the planet textures from the source code.

In [2]:
import re
colors = {
    "0": "",
    "1": "\\CHARCOLOR",
}
charsets_files = ["../iridisalpha/src/graphics/charset.asm"]
charsets = {}
for charsets_file in charsets_files:
    input_file = open(charsets_file,'r')
    charset_data = []
    curr_charset = []
    for l in input_file.readlines():
        if "CHARACTER" in l or "planet" in l:
            if charset_data:
                curr_charset += [charset_data]
            charset_data = []
        if "planet" in l or "characterSetData" in l:
            if curr_charset:
                charsets[charset_name] = curr_charset
            charset_name = l.strip()
            curr_charset = []
            continue

        m = re.findall(r"[0-1]{8}",l)
        if not m:
            continue
        bits = m[0]
        for i in range(0,8):
            bitpair = bits[i]
            charset_data += [colors[bitpair]]
    charsets[charset_name] = curr_charset
charsets["characterSetData"]

[['\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR'],
 ['',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '\\CHARCOLOR',
  '\\CHARCOLOR',
  '',
  '',
  '',
  '\\CHARCOLOR',
  '\\CHARC

In [3]:
def fillCharacter(c, i=0, BIT_WIDTH=8):
    """
    Create a latex output that fills the cells in an 8x8 cell
    in the appropriate position in the grid.
    """
    output = ""
    gh = BIT_WIDTH - 1
    for j,v in enumerate(c):
        if not v:
            continue
        x = (i * BIT_WIDTH) + ((j % BIT_WIDTH) if j >= (BIT_WIDTH - 1) else j)
        y = (gh - int(j/BIT_WIDTH)) if j >= gh else gh
        output += f'\t\\fill[{v}] ({x},{y}) rectangle ++ (1,1);\n'
    return output

In [32]:
def createTile(char, index, ref):
    """
    Create a tile in the tilesheet.
    """
    WIDTH = "0.8cm"
    header = f"""
    \\begin{{figure}}[H]
      {{
        \\setlength{{\\tabcolsep}}{{3.0pt}}
        \\setlength\cmidrulewidth{{\lightrulewidth}} % Make cmidrule = 
        \\begin{{adjustbox}}{{width={WIDTH},center}}
          \\begin{{tikzpicture}}
    """
    footer = f"""
          \end{{tikzpicture}}
        \end{{adjustbox}}
      }}\\caption*{{\{ref}}}
    \end{{figure}}
    """
    BIT_WIDTH = 8
    output = header + '\n'
    output += f"\\def\CHARCOLOR{{darkgray}}\n"
    
    grid = f"\\draw[step=1.0,gray,thin] (0,0) grid ({BIT_WIDTH},{BIT_WIDTH});"
    output += f"\t{grid}\n"

    output += fillCharacter(char)

    output += footer
    return output



Create a bunch of files of the form `planetCharset1_tilesheet.tex` that
contain a figure with the full tilesheet for that planet.

In [33]:
header = """
\\begin{figure}[H]                          
{                                          
  \\setlength{\\tabcolsep}{3.0pt}            
  \\setlength\cmidrulewidth{\lightrulewidth}
"""
figure = header
for i, c in enumerate(charsets["characterSetData"]):
    ref = f"${hex(0x00 + i)[2:].upper().zfill(2)}"
    output = createTile(c, i, ref)
    of_name = f"../src/titlescreen/tilesheets/charset_tilesheet_{ref}.tex"
    with open(of_name,'w') as output_file:
        output_file.write(output)

    figure += "\\begin{subfigure}{0.12\\textwidth}\n"
    figure += f"\\input{{tilesheets/charset_tilesheet_{ref}}}\n"
    figure += "\\end{subfigure}\n"

footer = f"""
}}\\caption[]{{Tilesheet: Font Character Set stored at \$2000}}
\\end{{figure}}
"""
figure += footer
of_name = f"../src/titlescreen/charset_tilesheet.tex"
with open(of_name,'w') as output_file:
    output_file.write(figure)


In [34]:
header = """
\\begin{figure}[H]                          
{                                          
  \\setlength{\\tabcolsep}{3.0pt}            
  \\setlength\cmidrulewidth{\lightrulewidth}
"""
figure = header
for i, c in enumerate(charsets["planetTextureCharset2"]):
    ref = f"${hex(0x60 + i)[2:].upper()}"
    output = createTile(c, i, ref)
    of_name = f"../src/titlescreen/tilesheets/charset_tilesheet_{ref}.tex"
    with open(of_name,'w') as output_file:
        output_file.write(output)

    figure += "\\begin{subfigure}{0.12\\textwidth}\n"
    figure += f"\\input{{tilesheets/charset_tilesheet_{ref}}}\n"
    figure += "\\end{subfigure}\n"

footer = f"""
}}\\caption[]{{Tilesheet: Main Game Font Character Set stored at \$2000}}
\\end{{figure}}
"""
figure += footer
of_name = f"../src/titlescreen/game_charset_tilesheet.tex"
with open(of_name,'w') as output_file:
    output_file.write(figure)
